In [2]:
import requests
import json


class INEGI_API:
    def __init__(self):
        self.url_base = "https://www.inegi.org.mx"
        self.headers = {
            'User-Agent': '####',
            'Accept-Language': '####',
            'Accept-Encoding': '####',
            'Connection': '####',
            'Accept-Encoding': 'identity'
        }

    def get_upc(self, clave):
        url = "https://www.inegi.org.mx/app/api/productos/interna_v2/componente/mapas/lista/resultados/"
        payload = {
            "muni": "",
            "loca": "",
            "tema": "193",
            "titg": "",
            "esca": 8,
            "form": "",
            "edic": "",
            "seri": "",
            "clave": clave,
            "rango": "",
            "busc": "",
            "tipoB": 2,
            "adv": False,
            "wordag": "",
            "mkeys": "",
            "mageo": "",
            "formIncl": "",
            "formExcl": "",
            "orden": 4,
            "desc": True,
            "pag": 0,
            "tam": 10
        }
        
        try:
            response = requests.post(url, headers=self.headers, json=payload)
            if response.status_code == 200:
                result = response.json()
                return result
            else:
                print(clave + "   Error en la solicitud. Código de estado:", response.status_code)
                return None
        except Exception as e:
            print(clave + "   Error:", str(e))
            return None

    def buscar_liga(self, json, tipo, ext):
        liga = {"liga": ""}
        if not json or not json['success']:
            return None
        
        for mapa in json['list']['mapas']:
            for formato in mapa["formatos"]:
                texto = formato['url']['valor'].lower()
                if tipo in texto and ext == 0 and "_b.zip" in texto:
                    liga["liga"] = formato['url']['valor']
                    return liga['liga']
                elif tipo in texto and ext == 1 and "_as.zip" in texto:
                    liga["liga"] = formato['url']['valor']
                    return liga['liga']
                elif tipo in texto and ext == 2 and "_gr.zip" in texto:
                    liga["liga"] = formato['url']['valor']
                    return liga['liga']
                elif tipo in texto and ext == 3 and "_t.zip" in texto:
                    liga["liga"] = formato['url']['valor']
                    return liga['liga']
                
        
        return None

    def download_dem(self, clave, tipo, ext, folder_output):

        salida = self.get_upc(clave)
        
        if salida:
            liga = self.buscar_liga(salida, tipo, ext)
            if liga:
                url = self.url_base + liga
                try:
                    response = requests.get(url, headers=self.headers)
                    if response.status_code == 200:
                        archivo_local = folder_output + clave + ".zip"
                        with open(archivo_local, 'wb') as archivo:
                            archivo.write(response.content)
                        print(clave + "   descargado y guardado")
                    else:
                        print(clave + "   Error en la solicitud. Código de estado:", response.status_code)
                except Exception as e:
                    print(clave + "   Error durante la descarga:", str(e))
            else:
                print(clave, "no tiene información")
        else:
            print(clave, "no se encontró en la base de datos")



In [7]:
import geopandas as gpd
import pandas as pd

poligono = gpd.read_file("inputs/cdmx.zip")
div10 = gpd.read_file("inputs/div10k.zip")
cartas = div10[div10.geometry.intersects(poligono.union_all())]

In [66]:
inegi_api = INEGI_API()

df_= pd.DataFrame()

for clave in cartas["clave10k"].to_list(): #[0:5]:
    respuesta = inegi_api.get_upc(clave)
    df = pd.json_normalize(respuesta['list']['mapas'], record_path='formatos', meta=['key', 'titulo', 'entidad', 'url', 'edicion', 'escala', 'clave_carta', 'datum', 'iin', 'af'])
    column_order = [
        'key', 'titulo', 'entidad', 'url', 'edicion', 'af', 'escala', 'clave_carta', 'datum', 'iin', 
        'upc', 'imagen', 'siglas', 'absoluto', 'dominio', 'folder', 'extension', 'datos_abiertos',
        'clave', 'peso', 'origen', 'web', 'control', 'url.valor', 'url.target', 'url.estilo', 'adicional.estilo'
    ]

    df = df[column_order]
    df_ = pd.concat([df_, df], ignore_index=True)
    df_['tipo'] = "superfie"
    df_['tipo'] = df_['titulo'].apply(lambda x: 'Terreno' if 'terreno' in x.lower() else 'Superficie')
    df_['clave_carta'] = df_['clave_carta'].str.lower()
    df_['id'] = df_['upc'].astype(str) + '_' + df_.index.astype(str)





In [60]:
#len(df_['clave_carta'].drop_duplicates().to_list())
def organizar(df, tipo, ext) 
df = pd.DataFrame()
df['clave_carta']= df_['clave_carta'].drop_duplicates()



In [62]:
print(len(df_['upc'].drop_duplicates()))
print(len(df_))

382
624


In [67]:
df_.head()

,key,titulo,entidad,url,edicion,af,escala,clave_carta,datum,iin,...,peso,origen,web,control,url.valor,url.target,url.estilo,adicional.estilo,tipo,id
0,702825770617,Modelos Digitales de Elevación de Alta Resoluc...,México,/app/biblioteca/ficha.html?upc=702825770617,2011,None,1:10 000,e14a29e2,ITRF92,False,...,4.04,1,False,False,/contenidos/productos/prod_serv/contenidos/esp...,_blank,,,Terreno,702825770617_0
1,702825770594,Modelos Digitales de Elevación de Alta Resoluc...,México,/app/biblioteca/ficha.html?upc=702825770594,2011,None,1:10 000,e14a29e2,ITRF92,False,...,4.11,1,False,False,/contenidos/productos/prod_serv/contenidos/esp...,_blank,,,Superficie,702825770594_1
2,702825770600,Modelos Digitales de Elevación de Alta Resoluc...,México,/app/biblioteca/ficha.html?upc=702825770600,2011,None,1:10 000,e14a29e2,ITRF92,False,...,9.79,1,False,False,/contenidos/productos/prod_serv/contenidos/esp...,_blank,,,Terreno,702825770600_2
3,702825770587,Modelos Digitales de Elevación de Alta Resoluc...,México,/app/biblioteca/ficha.html?upc=702825770587,2011,None,1:10 000,e14a29e2,ITRF92,False,...,9.84,1,False,False,/contenidos/productos/prod_serv/contenidos/esp...,_blank,,,Superficie,702825770587_3
4,702825770655,Modelos Digitales de Elevación de Alta Resoluc...,México,/app/biblioteca/ficha.html?upc=702825770655,2011,None,1:10 000,e14a29e3,ITRF92,False,...,2.83,1,False,False,/contenidos/productos/prod_serv/contenidos/esp...,_blank,,,Terreno,702825770655_4


In [ ]:
df_.to_csv("output/salida.csv")

In [40]:
inegi_api = INEGI_API()

df_= pd.DataFrame()

for clave in cartas["clave10k"].to_list()[0:5]:
    respuesta = inegi_api.get_upc(clave)
    #print(clave)
    print(respuesta['list']['mapas'])

[{'key': '702825770617', 'titulo': 'Modelos Digitales de Elevación de Alta Resolución LiDAR, con resolución de 5m. Terreno. GRID. E14A29E2', 'titulo_ing': '', 'entidad': 'México', 'url': '/app/biblioteca/ficha.html?upc=702825770617', 'edicion': 2011, 'formatos': [{'upc': '702825770617', 'imagen': '/img/ico/ico_grid.png', 'siglas': 'MB', 'absoluto': '', 'dominio': None, 'folder': None, 'extension': 'GRID', 'datos_abiertos': '', 'clave': 20, 'peso': 4.04, 'origen': 1, 'web': False, 'control': False, 'url': {'valor': '/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/imagen_cartografica/1_10_000/lidar/Terreno_GRID/702825770617_gr.zip', 'target': '_blank', 'estilo': ''}, 'adicional': {'estilo': ''}}], 'af': None, 'escala': '1:10 000', 'clave_carta': 'E14A29E2', 'serie': '', 'datum': 'ITRF92', 'resolucion': '', 'tipofoto': '', 'vuelo': '', 'iin': False, 'satelite': '', 'categoria': '', 'restriccion': '', 'toma': '', 'unidad_edo': '', 'contacto': '', 'coordenadas